# Dry Beans Models Project Notebook (Team 14)
##### Follow the steps in the README.md file to create and activate the virtual environment before running notebook cells

# Imports (Some imports may happen in later cells)

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from sklearn.model_selection import train_test_split

from ISLP import load_data, confusion_table
from ISLP.models import ModelSpec as MS

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
from sklearn.tree import (DecisionTreeClassifier as DTC, # Decision Tree for Classification
                          DecisionTreeRegressor as DTR, # Decision Tree for Regression
                          plot_tree, # Package for plotting trees
                          export_text)

from sklearn.ensemble import (RandomForestClassifier as RFC, # Random Forest for Classification
                              RandomForestRegressor as RFR, # Random Forest for Regression
                              GradientBoostingClassifier as GBC, # Boosting for Classification
                              GradientBoostingRegressor as GBR) # Boosting for Regression

from sklearn.metrics import (accuracy_score, # Method for finding Accuracy
                             confusion_matrix,
                             silhouette_score,
                             log_loss) # Method for finding the loss (log scale)

# Data pre-processing

In [5]:
# This cell converts our raw data file from .xlsx to .csv using pandas

# Load the xlsx file into a DataFrame
df = pd.read_excel('../raw_data/Dry_Bean_Dataset.xlsx')

# Save the DataFrame to a csv file
df.to_csv('../raw_data/Dry_Bean_Dataset.csv', index=False)

/Users/emery/Documents/GitHub/Dry_Beans_Models/venv/lib/python3.12/site-packages/openpyxl/packaging/core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
/Users/emery/Documents/GitHub/Dry_Beans_Models/venv/lib/python3.12/site-packages/openpyxl/packaging/core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


In [8]:
# Load the dataset
file_path = '../raw_data/Dry_Bean_Dataset.csv'

# Load the dataset
data = pd.read_excel(file_path)

# Display the first few rows of the dataset
print(data.head())

ValueError: Excel file format cannot be determined, you must specify an engine manually.

# Models